### This notebook was worked on in Google Colab. ###

Colab: https://colab.research.google.com/drive/1AdrQJ_AOlDVSw_6ve4Fr2DSkXxSYc8j-?usp=sharing

# Question 1: RAG Pipeline using BeyondLLM

## Setup

In [ ]:
# Install packages

!pip install beyondllm
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
  Using cached pydantic-2.9.1-py3-none-any.whl.metadata (146 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
Using cached pydantic-2.9.1-py3-none-any.whl (434 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
  Attemp

In [ ]:
# Import the required packages

from getpass import getpass
import os
from beyondllm import source,retrieve,embeddings,llms,generator
from beyondllm.embeddings import GeminiEmbeddings

# Used to securely store your API key
from google.colab import userdata

import gradio as gr

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# API KEYS
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEYL')

## Load Data

Access to data on drive: https://drive.google.com/file/d/1Gu0RygMX11nUwzE6Wjvj9f5w3IOoOcGP/view?usp=sharing

In [ ]:
# Fit the data from the pdf file
data_pdf = source.fit(path="/content/drive/MyDrive/Colab Notebooks/Bootcamp RAG & Agents - AI Planet/Challenge/Data/2402.17764v1.pdf", dtype="pdf", chunk_size=1024, chunk_overlap=100)

## Load Embedding Model

In [ ]:
# Load the embedding model
model_name_eb="models/embedding-001"
embed_model_ge = GeminiEmbeddings(api_key=GOOGLE_API_KEY, model_name=model_name_eb)

## Create Retriever

In [ ]:
# Initialize Retriever with Hybrid search
retriever_pdf = retrieve.auto_retriever(
    data=data_pdf,
    embed_model=embed_model_ge,
    type="hybrid",
    mode="OR",
    top_k=3)

## Load LLM

In [ ]:
# Load the LLM model

llm = llms.GeminiModel(
    model_name="gemini-pro",
    google_api_key=GOOGLE_API_KEY)

## Test Response

In [ ]:
# Test the Pipeline
def pipeline_test(query: str):
    pipeline = generator.Generate(question=query, retriever=retriever_pdf, llm=llm)
    response = pipeline.call()
    evaluate = pipeline.get_rag_triad_evals()
    response = f"\n\n{response}\n\nRAG Triad Evaluations:\n{evaluate}"
    return response

pipeline_test = pipeline_test("What is talked about in the document. Explain to me in five main points.")
print(pipeline_test)

LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.
Executing RAG Triad Evaluations...


1. **Activation-aware weight quantization (AWQ)**: A technique for compressing and accelerating large language models (LLMs) by quantizing their weights based on their activation patterns.
2. **Can a suit of armor conduct electricity?** A dataset for open book question answering that focuses on questions that require a broad understanding of the world.
3. **Pointer sentinel mixture models**: A type of neural network model for language generation that uses a mixture of pointers and sentinels to generate text.
4. **The LAMBADA dataset**: A dataset for word prediction that requires a broad discourse context.
5. **Llama 2**: An open and efficient foundation language model that can be fine-tuned for various tasks, including chat.

RAG Triad Evaluations:
Context relevancy Score: 1.0
This response does not meet the evaluation threshold. Consider refini

## Gradio App: Chat With PDF

In [ ]:
# Example Prompts
# You can use these prompts to test the app.
prompts_examples = ["Is there talk about LLMs in this document?",
                    "Is Artificial Intelligence discussed in this Document?",
                    "What is talked about in the document. Explain to me in three main points.",
                    "Tell me briefly about what is being discussed in: BitNet b1.58",
                    "Others",]

# Bot response function
def bot_app(query: str, chat_history: list) -> tuple:
        try:

            pipeline = generator.Generate(question=query, retriever=retriever_pdf, llm=llm)
            response = pipeline.call()
            evaluate = pipeline.get_rag_triad_evals()
            response = f"{response}\n\nRAG Triad Evaluations:\n{evaluate}"
            chat_history.append((query, response))
            return '', chat_history

        except Exception as e:
            chat_history.append((query, e))
            return '', chat_history

# Gradio Event Loop

CSS ="""
.contain { display: flex; flex-direction: column; height:80vh;}
#component-0 { height: 100%; }
#chatbot { flex-grow: 1; overflow: auto;}
"""
THEME = gr.themes.Default(primary_hue="green")

def get_demo():
    with gr.Blocks(css=CSS, theme=THEME) as demo:
        chatbot = gr.Chatbot(elem_id="chatbot")
        with gr.Row():
            msg = gr.Textbox(label="Human, enter your prompt!!!", placeholder="Enter text", scale=4,)

        clear = gr.ClearButton([msg, chatbot])
        msg.submit(bot_app, [msg, chatbot], [msg, chatbot], queue=False)
    return demo

# App Deployment
demo = get_demo()
demo.launch(share = True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://631f7f770ed4afae63.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.
Executing RAG Triad Evaluations...
LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.
Executing RAG Triad Evaluations...
LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.
Executing RAG Triad Evaluations...
LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.
Executing RAG Triad Evaluations...
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://631f7f770ed4afae63.gradio.live


# Question 2: AI Agent using OpenAGI

## Setup

In [ ]:
# Install packages

!pip install --upgrade openagi
!pip install --upgrade tavily-python
!pip install --upgrade langchain-google-genai
!pip install --upgrade yt-dlp youtube-search

  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.38
    Uninstalling langchain-core-0.2.38:
      Successfully uninstalled langchain-core-0.2.38
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 1.0.10 requires langchain-core<0.3,>=0.2.33, but you have langchain-core 0.1.52 which is incompatible.
  Using cached langchain_core-0.2.38-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.38-py3-none-any.whl (396 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
ERROR: pip's dependency resolver does not currently take in

In [ ]:
# Import the required packages

from openagi.actions.files import WriteFileAction, ReadFileAction
from openagi.actions.tools.ddg_search import DuckDuckGoNewsSearch
from openagi.actions.tools.webloader import WebBaseContextTool
from openagi.agent import Admin
from openagi.memory import Memory
from openagi.planner.task_decomposer import TaskPlanner
from openagi.worker import Worker
from rich.console import Console
from rich.markdown import Markdown
from openagi.llms.gemini import GeminiModel

from pydantic import Field
from openagi.actions.base import BaseAction

## LLM

In [ ]:
# Configure the LLM
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['Gemini_MODEL'] = 'gemini-1.5-flash'
os.environ['Gemini_TEMP'] = '0.8'

config = GeminiModel.load_from_env_config()
llm_agent = GeminiModel(config=config)

## Use Case in Spanish: Legislative Agent Spanish Version

This example shows how to create an OpenAGI agent with two workers (Search Specialist, Legislative Technical Specialist) with their corresponding tools, to autonomously research, write and review a Bill publication.

### Worker Agents

In [ ]:
# Worker Agents

# Initialize the researcher
researcher = Worker(
    role="Especialista de busqueda",
    instructions="Busque información de vanguardia sobre seguridad informatica. Trabajas en un grupo de expertos en seguridad informatica. Tienes la habilidad especial para analizar datos complejos y presentar información útil. Retorna la informacion en un resumen de una carilla en formato archivo de texto.",
    actions=[DuckDuckGoNewsSearch,
             WebBaseContextTool,
             WriteFileAction
    ], max_iterations=4
)

# Initialize the reviewer
reviewer = Worker(
    role="Especialista en Técnica Legislativa",
    instructions="Cree un blog en formato de Proyecto de Ley respetando la tecnica legislativa vigente utilizando la información investigada. Transforme conceptos complejos en contenido de legislacion convincente. Revise el contenido del blog para determinar su claridad, precisión gramatical y alineación con valores eticos y morales de la sociedad actual, refinelo para garantizar la perfección. Es un especialista meticuloso y atento a los detalles, que garantiza que cada contenido sea claro, y gramaticalmente perfecto. Retorna el blog creado y editado en formato archivo de texto.",
        actions=[ReadFileAction,
                 DuckDuckGoNewsSearch,
                 WebBaseContextTool,
                 WriteFileAction,
    ], max_iterations=4
)

### Admin

In [ ]:
# Declare the Admin object
admin = Admin(
    planner=TaskPlanner(human_intervene=False),
    memory=Memory(),
    llm=llm_agent,
)
# Assign sub-tasks to workers
admin.assign_workers([researcher, reviewer])

### Run the Admin

In [ ]:
# Run the admin object
response_agent = admin.run(
    query="Escribe una publicación de blog en formato de Proyecto de Ley (Máximo 20 articulos), sobre Seguridad Informática. Si es necesario puede escribir archivos en formato texto.",
    description="Realice un análisis integral de los últimos avances en Seguridad Informática en 2024. Identifique tendencias clave, tecnologías innovadoras y posibles impactos en la sociedad. Utilizando los conocimientos proporcionados, desarrolle una publicacion de blog que respete el formato solicitado y que destaque los avances más importantes. Tu publicación debe dirigirse a una audiencia conocedora de Tecnología y Legislación. Retorne la publicación de blog en formato archivo de texto.",
)



Tasks: [{'task_name': 'Buscar información sobre seguridad informática en 2024', 'description': 'Utiliza DuckDuckGo para buscar información sobre los últimos avances en seguridad informática en 2024. Enfócate en tendencias clave, tecnologías innovadoras y posibles impactos en la sociedad. Busca información que sea relevante para una audiencia conocedora de tecnología y legislación. Regresa el resumen de la información como un archivo de texto.', 'worker_id': 'c2c81371dc4e4b89898f5c6759533b2a'}, {'task_name': 'Crear el contenido del blog', 'description': 'Lee el archivo de texto con la información sobre seguridad informática. Utiliza esta información para crear un blog en formato de Proyecto de Ley (máximo 20 artículos). Asegúrate de que el contenido sea claro, preciso y fácil de entender para una audiencia conocedora de tecnología y legislación. El blog debe destacar los avances más importantes en seguridad informática en 2024. Regresa el blog como un archivo de texto.', 'worker_id': 

ERROR:root:JSON not extracted. Trying again.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/openagi/utils/extraction.py", line 70, in get_last_json
    return json.loads(last_json)
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 325 (char 324)


### Output

In [ ]:
# Output display
print("-" * 100)  # Separator
Console().print(Markdown(response_agent))

----------------------------------------------------------------------------------------------------


Proyecto de Ley: Avances en Seguridad Informática 2024                               

Artículo 1: Objeto                                                                                                 

El presente Proyecto de Ley tiene como objeto regular los avances más importantes en seguridad informática en el   
año 2024, con el fin de garantizar la protección de datos, sistemas y redes, y promover la ciberseguridad en la    
sociedad.                                                                                                          

Artículo 2: Alcance                                                                                                

Este Proyecto de Ley se aplicará a todos los sectores y actores que se involucren en el desarrollo, implementación 
y uso de tecnologías de la información y las comunicaciones (TIC) en el territorio nacional.                       

Artículo 3: Definiciones                                                                                           

Para los efectos de esta Ley, se entenderá por:                                                                    

 • Seguridad Informática: El conjunto de medidas técnicas y organizativas destinadas a proteger los sistemas de    
   información, los datos y las redes de las amenazas cibernéticas.                                                
 • Amenaza Cibernética: Cualquier acción o evento que pueda comprometer la seguridad de los sistemas de            
   información, los datos o las redes.                                                                             
 • Vulnerabilidad: Debilidad en un sistema de información que puede ser explotada por una amenaza cibernética.     
 • Ciberseguridad: El proceso de proteger los sistemas de información, los datos y las redes de las amenazas       
   cibernéticas, mediante la implementación de medidas preventivas, detectivas y correctivas.                      

Artículo 4: Avances en Seguridad Informática 2024                                                                  

4.1. Aumento de la concienciación sobre la ciberseguridad:                                                         

En 2024, se ha evidenciado un aumento significativo en la concienciación sobre la ciberseguridad en la sociedad.   
Organizaciones y gobiernos están impulsando campañas de educación y sensibilización para promover prácticas seguras
de navegación en internet y el uso responsable de las tecnologías.                                                 

4.2. Avances en la detección y prevención de ataques:                                                              

Se han implementado nuevas tecnologías y técnicas de detección y prevención de ataques, como la inteligencia       
artificial (IA), el aprendizaje automático (ML) y el análisis de comportamiento. Estas herramientas permiten       
identificar patrones sospechosos y responder a las amenazas de forma más eficiente.                                

4.3. Fortalecimiento de la infraestructura de seguridad:                                                           

Se ha observado un mayor enfoque en el fortalecimiento de la infraestructura de seguridad de las organizaciones. Se
están implementando medidas como la autenticación multifactor, la encriptación de datos y los sistemas de detección
de intrusiones (IDS) para proteger los sistemas y las redes.                                                       

4.4. Avances en la protección de datos personales:                                                                 

En 2024, se han promulgado nuevas regulaciones para proteger los datos personales, como el Reglamento General de   
Protección de Datos (GDPR) en la Unión Europea. Estas leyes obligan a las organizaciones a implementar medidas de  
seguridad para proteger la información de los usuarios.                                                            

4.5. Crecimiento del mercado de la ciberseguridad:                   

## Use Case in English: AI Teacher Agent

This example shows how to create an OpenAGI agent with three workers (Specialist Researcher, RAG PDF, Professor Agent) with their corresponding tools, to investigate, and formulate responses autonomously about Data Science and AI.

### Worker Agents

Access to data on drive: https://drive.google.com/file/d/1htvflad8yB9sH9G9U82CmwuXP-ERxoc-/view?usp=sharing

In [ ]:
# Worker Agents

# Custom component
class RAGBeyondLLM(BaseAction):
    """
    Use this action to extract content from the RAGBeyondLLM.

    """

    query: str = Field(default_factory=str,
                       description="User query, to get search results in the RAGBeyondLLM.")

    def execute(self):

        data_pdf = source.fit(path="/content/drive/MyDrive/Colab Notebooks/Bootcamp RAG & Agents - AI Planet/Challenge/Data/14500-brief-introduction-to-artificial-neural-networks-ensps.pdf", dtype="pdf", chunk_size=1024, chunk_overlap=100)
        embed_model_pdf = GeminiEmbeddings(api_key=GOOGLE_API_KEY, model_name="models/embedding-001")
        retriever_pdf = retrieve.auto_retriever(data=data_pdf, embed_model=embed_model_pdf, type="hybrid", mode="OR", top_k=3)
        llm = llms.GeminiModel(model_name="gemini-pro", google_api_key=GOOGLE_API_KEY)
        pipeline = generator.Generate(question=self.query, retriever=retriever_pdf, llm=llm)
        response = pipeline.call()
        return response

In [ ]:
# Query
query_ia = "What is Artificial Intelligence, and where can it be applied?"

# Initialize researcher_ia
researcher_ia = Worker(role="Specialist Researcher",
                       instructions=f"Research and analyze theory and concepts about Artificial Intelligence to answer the following question: {query_ia}. You have a knack for analyzing complex data and presenting useful information.",
                       actions=[DuckDuckGoNewsSearch],
                       max_iterations=2,
                       force_output=True
)

# Initialize researcher_ia_pdf
researcher_ia_pdf = Worker(role="RAG PDF",
                           instructions=f"Query: {query_ia}",
                           actions=[RAGBeyondLLM],
                           max_iterations=1,
                           force_output=True)

# Instruccion
instruccion_teacher ="""
Develop the requested concept using what was researched. Be creative and meticulous. He has five years of experience as a Professor of Data Science and AI. Make sure the concept is clear and perfect. Perform the requested task taking into account the information investigated and the following context data:
- Develop the concept of what was investigated.
- Use technical and scientific concepts.
- Maintain excellent grammar.
- You can place titles in bold.
- You can use emojis for more creative writing.
You must develop at least three paragraphs.
"""

# Initialize teacher_ia
teacher_ia = Worker(role="Professor Agent",
                      instructions=instruccion_teacher,
                      max_iterations=2,
                      force_output=True
)

### Admin

In [ ]:
# Declare the Admin object

admin_ia = Admin(planner=TaskPlanner(human_intervene=False),
                  memory=Memory(),
                  llm=llm_agent,
)
admin_ia.assign_workers([researcher_ia, researcher_ia_pdf, teacher_ia])

### Run the Admin

In [ ]:
# Run the admin object
response_agent_ia = admin_ia.run(
    query=query_ia,
    description="Analyze and investigate theory and concepts about Artificial Intelligence, you can use the web as a RAG pdf. Using the knowledge provided, develop a complete and clear response, respecting the research data and context. The request must be directed to Tertiary and University students of Data Science and AI.",
)



Tasks: [{'task_name': 'Research and Analyze AI Concepts', 'description': "Use DuckDuckGo to find and analyze relevant information about Artificial Intelligence and its applications. Provide a concise summary of the key concepts and applications that are most relevant to the question 'What is Artificial Intelligence, and where can it be applied?'  Return the summarized results as a string.", 'worker_id': '3579025845ca42b489d8cbc8d3e433c5'}, {'task_name': 'Extract Information from RAG PDF', 'description': "Use the RAGBeyondLLM action to extract information from the provided RAG PDF based on the query 'What is Artificial Intelligence, and where can it be applied?'.  Return the extracted information as a string.", 'worker_id': 'd702a8a3109043568e430456376ccdad'}, {'task_name': 'Develop the AI Concept', 'description': 'Use the information from the previous tasks to develop a clear and concise explanation of Artificial Intelligence and its applications.  Focus on providing a comprehensive 

LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.


### Output

Access to Log on drive: https://drive.google.com/drive/folders/1ktvvsOMkfFB2caPI8_djfAP70F94zWdd?usp=sharing

In [ ]:
# Output display
print("-" * 100)  # Separator
Console().print(Markdown(response_agent_ia))

----------------------------------------------------------------------------------------------------


Artificial Intelligence: A Paradigm Shift 🤖                                    

Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines capable of   
performing tasks that typically require human intelligence. These tasks encompass a wide range of capabilities,    
such as learning, problem-solving, decision-making, and even creative expression. AI systems leverage algorithms   
and data to mimic human cognitive functions, enabling them to understand, interpret, and respond to complex        
situations.                                                                                                        


                                     Applications of AI Across Industries: 🌐                                      

AI has permeated various industries, revolutionizing the way we live, work, and interact with the world. In        
healthcare, AI-powered diagnostic tools assist physicians in identifying diseases early and accurately. In finance,
AI algorithms detect fraudulent transactions and optimize investment strategies. In transportation, self-driving   
cars are being developed using AI to navigate roads autonomously. The applications of AI are vast and continue to  
expand, shaping the future of countless industries.                                                                


                                              Key Concepts in AI: 🧠                                               

At the heart of AI are fundamental concepts that drive its development and applications. Machine learning, a core  
component of AI, enables systems to learn from data without explicit programming. Deep learning, a subset of       
machine learning, utilizes artificial neural networks inspired by the human brain to process complex information.  
Natural language processing (NLP) empowers AI systems to understand and interact with human language, facilitating 
communication and information retrieval. Together, these concepts lay the groundwork for the advancement of AI and 
its transformative impact on society.

# Resources

* [API de Gemini](https://ai.google.dev/docs/gemini_api_overview)
* [Beyondllm](https://beyondllm.aiplanet.com/)
* [Gradio](https://www.gradio.app/docs)
* [OpenAGI](https://openagi.aiplanet.com/)